In [39]:
using Evolutionary
using Flux
using Flux: onehot, onecold, logitcrossentropy, onehotbatch, crossentropy
using MLDatasets
using Random
using Statistics
using MLJBase
using Printf
using BSON: @load # for load weights
using BSON: @save # for load weights
using Plots
using DelimitedFiles
import Evolutionary.initial_population
using Zygote
import Evolutionary.NonDifferentiable
import Base: copy, copyto!

In [40]:
features = Iris.features();
slabels = Iris.labels();
classes = unique(slabels)  # unique classes in the dataset
nclasses = length(classes) # number of classes
d, n = size(features)   

(4, 150)

In [41]:
data = [ (x, onehot(l, classes)) for (x, l) in zip(eachcol(features), slabels)]

150-element Vector{Tuple{SubArray{Float64, 1, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, Int64}, true}, Flux.OneHotArray{UInt32, 3, 0, 1, UInt32}}}:
 ([5.1, 3.5, 1.4, 0.2], [1, 0, 0])
 ([4.9, 3.0, 1.4, 0.2], [1, 0, 0])
 ([4.7, 3.2, 1.3, 0.2], [1, 0, 0])
 ([4.6, 3.1, 1.5, 0.2], [1, 0, 0])
 ([5.0, 3.6, 1.4, 0.2], [1, 0, 0])
 ([5.4, 3.9, 1.7, 0.4], [1, 0, 0])
 ([4.6, 3.4, 1.4, 0.3], [1, 0, 0])
 ([5.0, 3.4, 1.5, 0.2], [1, 0, 0])
 ([4.4, 2.9, 1.4, 0.2], [1, 0, 0])
 ([4.9, 3.1, 1.5, 0.1], [1, 0, 0])
 ([5.4, 3.7, 1.5, 0.2], [1, 0, 0])
 ([4.8, 3.4, 1.6, 0.2], [1, 0, 0])
 ([4.8, 3.0, 1.4, 0.1], [1, 0, 0])
 ⋮
 ([6.0, 3.0, 4.8, 1.8], [0, 0, 1])
 ([6.9, 3.1, 5.4, 2.1], [0, 0, 1])
 ([6.7, 3.1, 5.6, 2.4], [0, 0, 1])
 ([6.9, 3.1, 5.1, 2.3], [0, 0, 1])
 ([5.8, 2.7, 5.1, 1.9], [0, 0, 1])
 ([6.8, 3.2, 5.9, 2.3], [0, 0, 1])
 ([6.7, 3.3, 5.7, 2.5], [0, 0, 1])
 ([6.7, 3.0, 5.2, 2.3], [0, 0, 1])
 ([6.3, 2.5, 5.0, 1.9], [0, 0, 1])
 ([6.5, 3.0, 5.2, 2.0], [0, 0, 1])
 ([6.2, 3.4, 5.4, 2.3], [0, 0, 1

In [42]:
accuracy(model,x,y) = sum(onecold(model(x)) .== onecold(y))/size(x,2)
accuracy(xy, model) = sum( onecold(model(x)) .== onecold(y) for (x,y) in xy) /length(xy)

loss(model) = (x,y)->logitcrossentropy(model(x), y)
loss(model,x,y) = loss(model)(x, y)
loss(xy, model) = loss(model)(hcat(map(first,xy)...), hcat(map(last,xy)...))

loss (generic function with 3 methods)

In [43]:
fitness(m) = loss(data, m)

fitness (generic function with 1 method)

In [44]:
model = Chain(Dense(d, 15,relu), Dense(15, nclasses))

Chain(
  Dense(4, 15, relu),                   # 75 parameters
  Dense(15, 3),                         # 48 parameters
)                   # Total: 4 arrays, 123 parameters, 748 bytes.

In [56]:
model = Chain(Dense(d, nclasses))

Chain(
  Dense(4, 3),                          # 15 parameters
)

In [45]:
import Evolutionary.NonDifferentiable
NonDifferentiable(f, x::Chain) = NonDifferentiable{Real,typeof(x)}(f, f(x), deepcopy(x), [0,])
import Base: copy, copyto!

copy(ch::Chain) = deepcopy(ch)

function copyto!(l1::Dense{T}, l2::Dense{T}) where {T}
    copyto!(l1.W, l2.W)
    copyto!(l1.b, l2.b)
    l1
end

function copyto!(ch1::Chain, ch2::Chain)
    for i in 1:length(ch1.layers)
        copyto!(ch1.layers[i],ch2.layers[i])
    end
    ch1
end

copyto! (generic function with 140 methods)

In [46]:
import Evolutionary.initial_population
function initial_population(method::M, individual::Chain) where {M<:Evolutionary.AbstractOptimizer}
    θ, re = Flux.destructure(individual);
    [re(randn(length(θ))) for i in 1:Evolutionary.population_size(method)]
end

initial_population (generic function with 6 methods)

In [47]:
function gaussian_mlp(σ::Real = 1.0)
    vop = gaussian(σ)
    function mutation(recombinant::T) where {T <: Chain}        
        θ, re = Flux.destructure(recombinant)
        return re(convert(Vector{Float32}, vop(θ)))
    end
    return mutation
end

gaussian_mlp (generic function with 4 methods)

In [48]:
function uniform_mlp(m1::T, m2::T) where {T <: Chain}
    θ1, re1 = Flux.destructure(m1);
    θ2, re2 = Flux.destructure(m2);
    c1, c2 =average(θ1,θ2)
    return re1(c1), re2(c2)
end

uniform_mlp (generic function with 1 method)

In [49]:
gaussian_mlp(0.5)(model)

Chain(
  Dense(4, 15, relu),                   # 75 parameters
  Dense(15, 3),                         # 48 parameters
)                   # Total: 4 arrays, 123 parameters, 748 bytes.

In [98]:
algo = ES(
    initStrategy=IsotropicStrategy(3),

    mutation = gaussian,
    μ=10,
    λ=100,
    selection=:plus
)

ES(IsotropicStrategy{Float64}(1.0, 0.4082482904638631, 0.537284965911771), first, first, Evolutionary.gaussian, identity, 10, 10, 100, :plus)

In [108]:
import Evolutionary.gaussian
function gaussian(recombinant::Chain, s::IsotropicStrategy;
    rng::AbstractRNG=Random.GLOBAL_RNG)
    vop = gaussian(0.5)
    θ, re = Flux.destructure(recombinant)
    return re(convert(Vector{Float64}, vop(θ)))
return recombinant
end


gaussian (generic function with 7 methods)

In [109]:
res = Evolutionary.optimize(fitness, model, algo, opts)
model= Evolutionary.minimizer(res)

Chain(
  Dense(4, 3),                          # 15 parameters
)

In [110]:
accuracy(data, model)

0.3333333333333333

In [113]:
loss(data, model)

2.1255452956107526